In [1]:
from typing import Any, Sequence

In [3]:
def bin_search(a: Sequence, key:Any):
    
    pl = 0
    pr = len(a)-1
    
    while True:
        pc = (pl + pr)//2
        if a[pc]==key:
            return pc
        elif a[pc]<key:
            pl=pc+1
        else:
            pr = pc-1
        if pl>pr:
            break
            
    return -1


In [4]:
x=list(range(1,10))

In [5]:
bin_search(x,5)

4

In [6]:
bin_search(x,11)

-1

In [8]:
### chain으로 hash 구현

from __future__ import annotations
import hashlib

class Node:
    
    def __init__(self,key,value,next:Node):
        
        self.key = key
        self.value = value
        self.next = next

In [10]:
class ChainedHash:
    
    def __init__(self, capacity):
        
        self.capacity = capacity #해시 테이블의 크기 지정
        self.table = [None]*self.capacity #해시 테이블 선언
    
    def hash_value(self, key):
        #해시값 산출
        if isinstance(key,int):
            return key%self.capacity
        
        return (int(hashlib.sha256(str(key).encode()).hexdigest(),16)%self.capacity)
    
    def search(self,key):
        
        hash = self.hash_value(key) # 검색하는 key의 hash value
        p = self.table[hash] #노드를 주목
        
        while p is not None:
            if p.key == key:
                return p.value
            p = p.next
        return None
    
    def add(self,key,value):
        
        hash = self.hash_value(key)
        p = self.table[hash]
        
        while p is not None:
            if p.key ==key: 
                return False ###추가 실패, 이미 hash에 해당 값 존재
            
            p = p.next
        
        temp = Node(key,value,self.table[hash]) ###맨 앞에 추가, next를 self.table[hash]인 것으로 만듬
        self.table[hash] = temp
        return True
    
    
    def remove(self,key):
        
        hash = self.hash_value(key)  ###삭제할 key의 hash value
        p = self.table[hash]   ###node 주목
        pp = None   ### 바로 앞의 node를 주목
        
        while p is not None:
            if p.key == key:
                if pp is None:
                    self.table[hash] = p.next
                else:
                    pp.next = p.next #### p를 제거하고, 앞 node와 뒷 node를 연결해줌
                return True
            pp=p
            p = p.next
        
        return False
    
    def dump(self) :
        
        for i in range(self.capacity):
            p = self.table[i]
            print(i, end='')
            while p is not None:
                print(f'-> {p.key} ({p.value})',end='')
                p = p.next
            print()


In [13]:
### open addressing으로 hash 함수 구현

from __future__ import annotations
from enum import Enum
import hashlib

class Status(Enum):
    occupied = 0 ##데이터를 저장
    empty = 1 ## 비어있음
    deleted = 2 ##삭제완료
    
class Buckey:
    
    def __init__(self,key, value,stat):
        self.key = key
        self.value = value
        self.stat = stat
        
    def set(self,key,value,stat):
        self.key = key
        self.value = value
        self.stat = stat
    
    def set_status(self,stat):
        self.stat = stat

        
class OpenHash:
    
    def __init__(self,capacity):
        self.capacity = capacity
        self.table = [Bucket()]*self.capacity ## hash table 생성
    
    def hash_value(self,key):
        if isinstance(key,int):
            return key%self.capacity
        return (int(hashlib.sha256(str(key).encode()).hexdigest(),16)%self.capacity)
    
    def rehash_value(self,key):
        return (self.hash_value(key)+1)%self.capacity
    
    def search_node(self,key):
        hash = self.hahs_value(key)
        p = self.table[hash]
        
        for i in range(self.capacity):
            if p.stat == Status.empty:
                break
            elif p.stat == Status.occupied and p.key == key:
                return p
            hash = self.rehash_value(hash)
            p = self.table[hash]
        
        return None
    
    def search(self,key):
        p = self.search_node(key)
        if p is not None:
            return p.value
        else:
            return None
        
    
    def add(self,key,value):
        if self.search(key) is not None:
            return False ##이미 등록된 key
        
        hash = self.hash_value(key)
        p = self.table[hash]
        for i in range(self.capacity):
            if p.stat == Status.empty or p.stat == Status.deleted:
                self.table[hash] = Bucket(key,value, Status.occupied)
                return True
            hash = self.rehash_value(hash)
            p = self.table[hash]
        
        return False
        
    
    def remove(self,key):
        p=self.search_node(key)
        if p is None:
            return False
        p.set_status(Status.deleted)
        return True
    
    def dump(self):
        for i in range(self.capacity):
            print(f'{i:2}',end='')
            if self.table[i].stat == Status.occupied:
                print(f'{self.table[i].key}({self.table[i].value})')
            elif self.table[i].stat == Status.empty:
                print('--미등록--')
            elif self.table[i].stat == Status.deleted:
                print('--삭제완료--')